# What is ETL?



In [45]:
import requests
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

## Extract

In [46]:
print("\n[1] Extracting data from CoinGecko API...")

url = "https://api.coingecko.com/api/v3/simple/price"
params = {
    "ids": "bitcoin,ethereum",
    "vs_currencies": "usd,eur,gbp"
}

response = requests.get(url, params=params)
data = response.json()

print("Raw API response:")
print(data)


[1] Extracting data from CoinGecko API...
Raw API response:
{'bitcoin': {'usd': 112158, 'eur': 96185, 'gbp': 83427}, 'ethereum': {'usd': 4470.66, 'eur': 3833.97, 'gbp': 3325.44}}


## Transform

In [47]:
print("\n[2] Transforming data...")

df = pd.DataFrame([{
    "timestamp": datetime.now(),
    "BTC_USD": data["bitcoin"]["usd"],
    "BTC_EUR": data["bitcoin"]["eur"],
    "BTC_GBP": data["bitcoin"]["gbp"],
    "ETH_USD": data["ethereum"]["usd"],
    "ETH_EUR": data["ethereum"]["eur"],
    "ETH_GBP": data["ethereum"]["gbp"]
}])

# Example transform: add BTC/ETH ratio
df["BTC_to_ETH_USD"] = df["BTC_USD"] / df["ETH_USD"]

print("Data after transformation:")
print(df)


[2] Transforming data...
Data after transformation:
                   timestamp  BTC_USD  BTC_EUR  BTC_GBP  ETH_USD  ETH_EUR  \
0 2025-09-03 16:28:34.559181   112158    96185    83427  4470.66  3833.97   

   ETH_GBP  BTC_to_ETH_USD  
0  3325.44       25.087571  


## Load

In [48]:
print("\n[3] Loading data into SQLite database...")

engine = create_engine("sqlite:///crypto_prices.db")

# Append so we keep history over time
df.to_sql("crypto_prices", engine, if_exists="append", index=False)

print("Data successfully loaded into 'crypto_prices' table.")



[3] Loading data into SQLite database...
Data successfully loaded into 'crypto_prices' table.


In [ ]:
print("\n[4] Verifying database contents...")

check = pd.read_sql(
    "SELECT * FROM crypto_prices ORDER BY timestamp DESC", 
    engine
)
print(check)

print("\nETL pipeline finished successfully!")



[4] Verifying database contents (last 5 rows)...
                    timestamp  BTC_USD  BTC_EUR  BTC_GBP  ETH_USD  ETH_EUR  \
0  2025-09-03 16:28:34.559181   112158    96185    83427  4470.66  3833.97   
1  2025-09-03 16:28:31.383368   112158    96185    83427  4470.66  3833.97   
2  2025-09-03 16:28:18.462919   112158    96185    83427  4470.66  3833.97   
3  2025-09-03 12:03:16.128413   112267    96160    83482  4482.36  3839.24   
4  2025-09-03 12:03:14.743631   112267    96160    83482  4482.36  3839.24   
5  2025-09-03 12:03:02.867696   112267    96160    83482  4482.36  3839.24   
6  2025-09-03 12:02:40.123462   112267    96160    83482  4482.36  3839.24   
7  2025-09-03 11:59:26.764711   112211    96085    83418  4483.34  3839.03   

   ETH_GBP  BTC_to_ETH_USD  
0  3325.44       25.087571  
1  3325.44       25.087571  
2  3325.44       25.087571  
3  3333.09       25.046404  
4  3333.09       25.046404  
5  3333.09       25.046404  
6  3333.09       25.046404  
7  3332.90     